In [1]:
import os
import cv2
import kagglehub
import numpy as np
from PIL import Image
from typing import Tuple, Optional, Union
from sklearn.feature_extraction import image

import torch
import torch.nn as nn
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset

from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader
from torch_geometric.utils import negative_sampling
from torch_geometric.nn import GCNConv, VGAE, global_mean_pool, global_max_pool, global_add_pool

from torchvision import transforms

from tqdm import tqdm
from tqdm.contrib import tmap

from lib.lib import SignatureDataset, image_to_graph

In [2]:
class GNNEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, latent_dim):
        super(GNNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv_mu = GCNConv(hidden_channels, latent_dim)
        self.conv_logvar = GCNConv(hidden_channels, latent_dim)

    def forward(self, x, edge_index):
        # Step 1: Aggregate node features from neighbors
        x = F.relu(self.conv1(x, edge_index))

        # Step 2: Output mean and log variance
        mu = self.conv_mu(x, edge_index)
        logvar = self.conv_logvar(x, edge_index)

        return mu, logvar

In [3]:
# Hyperparameters
learning_rate = 1e-3
w_d = 1e-5
batch_size = 32
epochs = 50
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

device

device(type='cpu')

In [4]:
def dataset_path():
    path = kagglehub.dataset_download("akashgundu/signature-verification-dataset")
    return os.path.join(path, 'extract')

def transform(**kwargs):
    return transforms.Compose([
        transforms.Grayscale(num_output_channels=kwargs['num_output_channels']),
        transforms.Resize(kwargs['resize']),
        transforms.ToTensor(),
    ])

dataset = SignatureDataset(
    root_dir=dataset_path(),
    transform=transform(num_output_channels=1, resize=(64, 64))
)

Loaded 14626 signature images (genuine + forged)


In [5]:
next(iter(dataset))

tensor([[[0.9569, 0.9804, 0.9804,  ..., 0.9765, 0.9804, 0.9843],
         [0.9804, 0.9882, 1.0000,  ..., 0.9961, 1.0000, 0.9961],
         [0.9882, 0.9961, 1.0000,  ..., 1.0000, 1.0000, 0.9961],
         ...,
         [0.9843, 0.9961, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [0.9804, 0.9961, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [0.9686, 0.9922, 1.0000,  ..., 0.9961, 0.9961, 1.0000]]])

In [6]:
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = total_size - train_size
train_dataset, val_dataset = random_split(
    dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)
print(f"Dataset sizes - Train: {train_size}, Validation: {val_size}")

Dataset sizes - Train: 11700, Validation: 2926


In [7]:
train_dataset[2]

tensor([[[0.6392, 0.6392, 0.6353,  ..., 1.0000, 1.0000, 1.0000],
         [0.7686, 0.7686, 0.7686,  ..., 1.0000, 1.0000, 1.0000],
         [0.9922, 0.9922, 0.9922,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]]])

In [8]:
def graph_converter(data):
    d_graph = image_to_graph(data)
    return d_graph   

# Convert training dataset
train_graph = list(tmap(graph_converter, train_dataset, desc="Train Graphs"))

# Convert validation dataset
val_graph = list(tmap(graph_converter, val_dataset, desc="Val Graphs"))

print("Train graphs:", len(train_graph))
print("Val graphs:", len(val_graph))

Train Graphs:   0%|          | 0/11700 [00:00<?, ?it/s]

Val Graphs:   0%|          | 0/2926 [00:00<?, ?it/s]

Train graphs: 11700
Val graphs: 2926


In [10]:
train_loader = DataLoader(
    train_graph,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0
)

val_loader = DataLoader(
    val_graph,
    batch_size=256,
    shuffle=False,
    num_workers=0
)

next(iter(train_loader))

DataBatch(x=[131072, 3], edge_index=[2, 516096], batch=[131072], ptr=[33])

In [11]:
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.utils import negative_sampling
from datetime import datetime

# Create unique writer for each run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
writer = SummaryWriter(f'runs/gae_experiment_{timestamp}')

def train_step(model, optimizer, data_list, beta=1.0):
    """
    Training step for a list of graph Data objects.
    Each graph is trained independently.
    """
    model.train()
    total_loss = 0.0

    for data in tqdm(data_list, desc="Training", leave=False):
        data = data.to(device)
        optimizer.zero_grad()

        # Encode nodes into latent space
        z = model.encode(data.x, data.edge_index)

        # Reconstruction + KL loss
        recon_loss = model.recon_loss(z, data.edge_index)
        kl_loss = (1 / data.num_nodes) * model.kl_loss()
        loss = recon_loss + beta * kl_loss

        # Backpropagation
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(data_list)
    return avg_loss

@torch.no_grad()
def val_step(model, val_list, beta=1.0):
    """
    Validation step for a list of graph Data objects.
    No gradient updates are performed.
    """
    model.eval()
    total_loss = 0.0

    for data in tqdm(val_list, desc="Validating", leave=False):
        data = data.to(device)
        z = model.encode(data.x, data.edge_index)

        recon_loss = model.recon_loss(z, data.edge_index)
        kl_loss = (1 / data.num_nodes) * model.kl_loss()
        loss = recon_loss + beta * kl_loss

        total_loss += loss.item()

    avg_loss = total_loss / len(val_list)
    return avg_loss

In [12]:
input_dim = next(iter(train_graph)).x.shape[1]
hidden_dim = 64
latent_dim = 128
# epochs = 500
epochs = 500

In [13]:
# Initialize model
model = VGAE(GNNEncoder(in_channels=input_dim, hidden_channels=hidden_dim, latent_dim=latent_dim)).to(device)

# Create optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
best_val_loss = float("inf")

def train_vgae(model, optimizer, train_loader, val_loader, epochs=100, beta=1.0, log_dir="runs/vgae_signature"):
    # Initialize TensorBoard writer
    writer = SummaryWriter(log_dir)

    global best_val_loss
    patience = 10  # early stopping patience
    wait = 0

    for epoch in range(1, epochs + 1):
        train_loss = train_step(model, optimizer, train_loader, beta)
        val_loss = val_step(model, val_loader, beta)

        # Log to TensorBoard
        writer.add_scalar("Loss/Train", train_loss, epoch)
        writer.add_scalar("Loss/Validation", val_loss, epoch)
        writer.add_scalar("KL_Beta", beta, epoch)

        print(f"Epoch {epoch:03d} | Train: {train_loss:.4f} | Val: {val_loss:.4f}")

        # Early stopping based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            wait = 0
            torch.save(model.state_dict(), os.path.join(log_dir, "best_vgae_model.pth"))
        else:
            wait += 1
            if wait >= patience:
                print("⏹️ Early stopping triggered!")
                break

    writer.close()
    return model
    print("✅ Training finished. Best model saved.")

In [ ]:
vgae = train_vgae(model, optimizer, train_loader, val_loader, epochs=epochs, beta=1.0, log_dir="runs/vgae_signature")

Training:   0%|                                                                                | 0/366 [00:00<?, ?it/s]

In [ ]:
torch.save(model.state_dict(), 'VGAE_Model.pt')

In [16]:
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': 442,
    'val_loss': 0.7914,
}, 'VGAE_Model_All_Saved.pt')